<a href="https://colab.research.google.com/github/nirbarash88/node-js-exprees-server-react-client/blob/master/Part_B_of_Final_Project_of_Image_Processing_DataSet_Creation_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import cv2
import numpy as np
import random
from google.colab import drive

# Mount Google Drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
# Helper function to get intensity range for each shape type
def get_intensity_range(shape_type):
    if shape_type == "circle":
        return 26, 50
    elif shape_type == "rectangle":
        return 51, 75
    elif shape_type == "triangle":
        return 76, 100
    elif shape_type == "trapezoid":
        return 101, 125
    elif shape_type == "ellipse":
        return 126, 150
    elif shape_type == "square":
        return 151, 175
    elif shape_type == "half_circle":
        return 176, 200
    elif shape_type == "pentagon":
        return 201, 225
    elif shape_type == "hexagon":
        return 226, 250

# Function to generate a random shape and associated metadata
def generate_shape(image):
    h, w = image.shape[:2]

    # Generate random shape
    shape_type = random.choice(["circle", "rectangle", "triangle", "trapezoid", "ellipse", "square", "half_circle", "pentagon", "hexagon"])
    intensity_range_min, intensity_range_max = get_intensity_range(shape_type)
    intensity_range = random.randint(intensity_range_min, intensity_range_max)
    color = random.randint(intensity_range - 25, min(intensity_range, 255))
    size = random.randint(10, min(h, w) // 4)
    x, y = random.randint(size, w - size), random.randint(size, h - size)

    # Draw shape on the image
    if shape_type == "circle":
        cv2.circle(image, (x, y), size, color, -1)
    elif shape_type == "rectangle":
        cv2.rectangle(image, (x - size, y - size), (x + size, y + size), color, -1)
    elif shape_type == "triangle":
        points = np.array([[x, y - size], [x - size, y + size], [x + size, y + size]])
        cv2.drawContours(image, [points], 0, color, -1)
    elif shape_type == "trapezoid":
        points = np.array([[x - size, y + size], [x + size, y + size], [x - size // 2, y - size], [x + size // 2, y - size]])
        cv2.drawContours(image, [points], 0, color, -1)
    elif shape_type == "ellipse":
        cv2.ellipse(image, (x, y), (size, size // 2), 0, 0, 360, color, -1)
    elif shape_type == "square":
        cv2.rectangle(image, (x - size, y - size), (x + size, y + size), color, -1)
    elif shape_type == "half_circle":
        cv2.ellipse(image, (x, y), (size, size // 2), 0, -180, 0, color, -1)
    elif shape_type == "pentagon":
        side_length = size
        angle = 360 / 5
        pentagon_points = []
        for i in range(5):
            pentagon_points.append((int(x + side_length * np.cos(np.radians(i * angle))), int(y + side_length * np.sin(np.radians(i * angle)))))
        cv2.fillPoly(image, [np.array(pentagon_points)], color)
    elif shape_type == "hexagon":
        side_length = size
        angle = 360 / 6
        hexagon_points = []
        for i in range(6):
            hexagon_points.append((int(x + side_length * np.cos(np.radians(i * angle))), int(y + side_length * np.sin(np.radians(i * angle)))))
        cv2.fillPoly(image, [np.array(hexagon_points)], color)

    # Generate metadata for the shape
    metadata = {"shape_type": shape_type, "color": color, "size": size, "position": (x, y)}

    # Return metadata
    return metadata

# Function to generate a random line and associated metadata
def generate_line(image):
    h, w = image.shape[:2]

    # Generate random line
    x1, y1 = random.randint(0, w), random.randint(0, h)
    x2, y2 = random.randint(0, w), random.randint(0, h)

    # Ensure x1 and x2 are not equal to avoid division by zero
    while x1 == x2:
        x2 = random.randint(0, w)

    # Calculate coefficients of the line equation (y = mx + c)
    m = (y2 - y1) / (x2 - x1)
    c = y1 - m * x1

    # Generate random color for the line
    color = random.randint(0, 25)

    # Generate metadata for the line
    metadata = {"shape_type": "line", "color": color, "a": m, "b": c}

    # Draw line on the image
    cv2.line(image, (x1, y1), (x2, y2), color, 1)

    # Return metadata
    return metadata

import json
# Decided using json to help the classification process in the next notebook.
# Parsing JSON into MetaData Object is easier than plain text

# Function to generate an image with shapes and a line
def generate_image():
    image = np.zeros((128, 128), dtype=np.uint8)
    shape_metadata = []

    # Randomize number of shapes between 0 and 9
    num_shapes = random.randint(0, 9)

    # Generate shapes and collect metadata
    for _ in range(num_shapes):
        shape_metadata.append(generate_shape(image))

    # Generate line and collect metadata
    line_metadata = generate_line(image)

    # Create dictionary to hold image and metadata
    data = {
        "shape_metadata": shape_metadata,
        "line_metadata": line_metadata
    }

    # Convert dictionary to JSON string
    json_data = json.dumps(data)

    return image, json_data



In [ ]:
import tempfile
import shutil
import os

# Generate and save images locally
temp_dir = tempfile.mkdtemp()
num_train = 5600
num_test = 1400

for subset_folder in ["train", "test"]:
    os.makedirs(os.path.join(temp_dir, subset_folder), exist_ok=True)
    iterations = 0
    if(subset_folder == "train"):
      iterations = num_train
    else:
      iterations = num_test
    for i in range(iterations):
        image, json_metadata = generate_image()
        filename = f"{i}.jpg"
        print("Starting generation for: " + temp_dir +"/"+ subset_folder+"/"+filename)
        cv2.imwrite(f"{temp_dir}/{subset_folder}/{filename}", image)
        # Save metadata
        with open(f"{temp_dir}/{subset_folder}/{filename}.txt", "w") as f:
            f.write(json_metadata)

print("Image generation completed.")

Streaming output truncated to the last 5000 lines.
Starting generation for: /tmp/tmpom7qn3jo/train/2001.jpg
Starting generation for: /tmp/tmpom7qn3jo/train/2002.jpg
Starting generation for: /tmp/tmpom7qn3jo/train/2003.jpg
Starting generation for: /tmp/tmpom7qn3jo/train/2004.jpg
Starting generation for: /tmp/tmpom7qn3jo/train/2005.jpg
Starting generation for: /tmp/tmpom7qn3jo/train/2006.jpg
Starting generation for: /tmp/tmpom7qn3jo/train/2007.jpg
Starting generation for: /tmp/tmpom7qn3jo/train/2008.jpg
Starting generation for: /tmp/tmpom7qn3jo/train/2009.jpg
Starting generation for: /tmp/tmpom7qn3jo/train/2010.jpg
Starting generation for: /tmp/tmpom7qn3jo/train/2011.jpg
Starting generation for: /tmp/tmpom7qn3jo/train/2012.jpg
Starting generation for: /tmp/tmpom7qn3jo/train/2013.jpg
Starting generation for: /tmp/tmpom7qn3jo/train/2014.jpg
Starting generation for: /tmp/tmpom7qn3jo/train/2015.jpg
Starting generation for: /tmp/tmpom7qn3jo/train/2016.jpg
Starting generation for: /tmp/tmpom7q

In [ ]:
# Copy images to Google Drive
dataset_folder = "/content/drive/My Drive/Part_B_DataSet"  # Destination folder in Google Drive
for subset_folder in ["train", "test"]:
    print("Started Copying: " + subset_folder)
    destination_folder = os.path.join(dataset_folder, subset_folder)
    try:
        shutil.copytree(f"{temp_dir}/{subset_folder}", destination_folder)
        print("Finished Copying: " + subset_folder)
    except FileExistsError:
        print(f"Folder {destination_folder} already exists. Skipping copying.")

print("Images copied to Google Drive.")

Started Copying: train
Folder /content/drive/My Drive/Part_B_DataSet/train already exists. Skipping copying.
Started Copying: test
Folder /content/drive/My Drive/Part_B_DataSet/test already exists. Skipping copying.
Images copied to Google Drive.


In [ ]:
import shutil
import os
import zipfile
from google.colab import files

def copy_folder_to_local(src_folder, dest_zip_file):
    """
    Copies a folder from Google Colab to the local machine by compressing it into a ZIP archive.

    Args:
    - src_folder (str): Path to the source folder in Google Colab.
    - dest_zip_file (str): Path to the destination ZIP file on the local machine.
    """
    # Ensure that the source folder exists
    if not os.path.exists(src_folder):
        print(f"Source folder '{src_folder}' does not exist.")
        return

    # Create a ZIP archive of the source folder
    with zipfile.ZipFile(dest_zip_file, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Walk through the source folder and add each file to the ZIP archive
        for root, dirs, files in os.walk(src_folder):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, os.path.relpath(file_path, src_folder))

    print(f"Folder '{src_folder}' compressed to '{dest_zip_file}'.")


src_folder = temp_dir
dest_zip_file = '/content/images.zip'
copy_folder_to_local(src_folder, dest_zip_file)
files.download(dest_zip_file)

Folder '/tmp/tmpom7qn3jo' compressed to '/content/images.zip'.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>